Notebook showing usage of the data detective.

In [13]:
# To import detective from relative folder
import detective.core as detective
import detective.functions as functions
import pandas as pd

Supply the db_url as described in the [recorder integration docs](https://www.home-assistant.io/integrations/recorder/).

In [14]:
db_url = "sqlite://///Users/robin/Github/HASS-data-detective/tests/test.db"

In [15]:
db = detective.HassDatabase(db_url) # To init without fetching entities fetch_entities=False

Successfully connected to database sqlite://///Users/robin/Github/HASS-data-detective/tests/test.db
There are 9 entities with data


Alternatively, detective can discover your database credentials

In [16]:
# db = detective.db_from_hass_config()

Entities are listed on an attribute

In [17]:
db.entities

['persistent_notification.config_entry_discovery',
 'sensor.sun_next_dawn',
 'sensor.sun_next_dusk',
 'sensor.sun_next_midnight',
 'sensor.sun_next_noon',
 'sensor.sun_next_rising',
 'sensor.sun_next_setting',
 'sun.sun',
 'zone.home']

Use `fetch_all_data_of()` to get data for a single entity

In [18]:
df = db.fetch_all_data_of(('zone.home',))


            WITH combined_states AS (
                SELECT states.state, states.last_updated_ts, states_meta.entity_id
                FROM states
                JOIN states_meta
                ON states.metadata_id = states_meta.metadata_id
            )
            SELECT *
            FROM combined_states
            WHERE 
                entity_id IN ('zone.home')
            AND
                state NOT IN ('unknown', 'unavailable')
            ORDER BY last_updated_ts DESC
        LIMIT 50000
The returned Pandas dataframe has 1 rows of data.


In [19]:
df.head()

,state,last_updated_ts,entity_id
0,0,1.680324e+09,zone.home


Use `fetch_all_sensor_data()` to fetch all your sensor data into a pandas dataframe in memory. 

**Note** that by default the number of states returned is limited but this can optionally be over-ridden as shown below. 

In [20]:
df = db.fetch_all_sensor_data(limit=100000)


            SELECT states.state, states.last_updated_ts, states_meta.entity_id
            FROM states
            JOIN states_meta
            ON states.metadata_id = states_meta.metadata_id
            WHERE
                states_meta.entity_id  LIKE '%sensor%'
            AND
                states.state NOT IN ('unknown', 'unavailable')
            ORDER BY last_updated_ts DESC
        LIMIT 100000
The returned Pandas dataframe has 6 rows of data.


Take a minute to read about the differences between [wide-form and long-form data](https://altair-viz.github.io/user_guide/data.html#long-form-vs-wide-form-data). The Pandas dataframe we have is in long form.

In [21]:
df

,state,last_updated_ts,entity_id
0,2023-04-01T18:07:00+00:00,1.680324e+09,sensor.sun_next_setting
1,2023-04-01T06:00:49+00:00,1.680324e+09,sensor.sun_next_rising
2,2023-04-01T12:04:03+00:00,1.680324e+09,sensor.sun_next_noon
3,2023-04-02T00:03:37+00:00,1.680324e+09,sensor.sun_next_midnight
4,2023-04-01T18:28:08+00:00,1.680324e+09,sensor.sun_next_dusk
5,2023-04-01T05:39:42+00:00,1.680324e+09,sensor.sun_next_dawn


It is necessary to format the data before we can plot it, and detective provides a functions to assist

In [22]:
df = functions.format_dataframe(df)

In [23]:
df.head()

,state,last_updated_ts,entity_id


It is straightforward to create your own features, for example to add a `day_of_week` column

In [24]:
df['day_of_week'] = df['last_updated_ts'].apply(lambda x : x.dayofweek)
df['is_temperature'] = df['entity_id'].apply(lambda x : 'temperature' in x)

From here you can begin plotting